In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/Project/Media

/content/gdrive/My Drive/Project/Media


In [0]:
ls

Dataset/  Graph/  logs/  Model/  Preprocess.ipynb  train.ipynb


In [7]:
!git commit -m "Finish Text Training"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@013164c41df4.(none)')


In [0]:
!git config --global user.name jiadonglee

In [0]:
# -*- coding: utf-8 -*-
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.feature_extraction import text
from sklearn import decomposition, ensemble
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import numpy as np
import string, jieba, re, os, sys
from keras import layers, models, optimizers

Using TensorFlow backend.


In [0]:
Df_text_True = pd.read_json("Dataset/text_modal/truth.json", lines=True)
Df_text_False = pd.read_json("Dataset/text_modal/rumor.json", lines=True)
Df_text = Df_text_True.content.append(Df_text_False.decision)

In [0]:
Df_text_True.head()

,_position,comment,containsMusic,containsUrl,containsVideo,content,faces,forward,hotrate,isClue,isOrigin,isPiyao,mid,name,piclist,praise,sourcePlatform,time,url,userCertify,userDescription,userFanCount,userFollowCount,userGender,userId,userLocation,userWeiboCount,userurl,weiboLocation,weibo_comments
0,"[0, 10]",0.0,0.0,0.0,0.0,\n\t\t傻逼媒体//【济宁学院男生因分手问题将女友捅死 两人均19岁】 http://t...,None,0.0,0,0.0,True,False,3955208083486498,黎奥黎奥,[http://ww3.sinaimg.cn/large/005Cif4hjw1f23mhh...,0,手机新浪网,1458478554000,http://weibo.com/5146512757/DnadieCZQ,0,开始,61.0,143.0,male,5146512757,辽宁沈阳,508.0,http://weibo.com/u/5146512757?refer_flag=10010...,None,[]
1,"[0, 18]",0.0,0.0,0.0,0.0,\n\t\t正在图书馆一惊一乍的看这些恐怖电影解析，突然听到图书管理员跟男盆友打电话说济宁学...,None,0.0,0,0.0,True,False,3954128994859460,你看不见我的自由自在s,[http://ww4.sinaimg.cn/large/005NcfOJjw1f206jw...,3,荣耀3X畅玩版,1458221278000,http://weibo.com/5307625365/DmI8Pkoao,0,微博认证：微博气人博主,244.0,710.0,female,5307625365,山东青岛,1307.0,http://weibo.com/u/5307625365?refer_flag=10010...,None,[]
2,"[1, 10]",0.0,NaN,NaN,NaN,\n\t\t发表了博文《铁路部门“指尖餐厅”旅客赞不绝口》旅客乘坐高铁就餐更方便啦!日前，上...,NaN,0.0,0,NaN,True,False,3958712869287921,苹果味酸奶,[http://s6.sinaimg.cn/thumb180/6c0c0bb3tdf7bde...,0,新浪博客,1459314161000,http://weibo.com/1975531711/DoDoaCYdb,0,None,85.0,132.0,female,1975531711,河北保定,113.0,http://weibo.com/u/1975531711?refer_flag=10010...,NaN,[]
3,"[1, 17]",0.0,NaN,NaN,NaN,\n\t\t发表了博文《清明小长假期间长三角铁路预计日均发送旅客200万人》3月19日至4月...,NaN,0.0,0,NaN,True,False,3955603619325730,不管-你是谁,[http://s6.sinaimg.cn/thumb180/6c0c0bb3tdf7bde...,0,新浪博客,1458572856000,http://weibo.com/3721485623/DnkvfD830,0,有话就要说,7.0,5.0,female,3721485623,江苏南京,908.0,http://weibo.com/u/3721485623?refer_flag=10010...,NaN,[]
4,"[1, 28]",0.0,NaN,NaN,NaN,\n\t\t发表了博文《原创微小说+++他走了》他发来微信说，他走了，走到哪里算哪里，他真心...,NaN,0.0,0,NaN,True,False,3958668456199372,束菊艳,[http://h5.sinaimg.cn/upload/2015/07/13/6/time...,0,新浪博客,1459303571000,http://weibo.com/5166057412/DoCexq0JS,0,None,17.0,5.0,female,5166057412,浙江嘉兴,201.0,http://weibo.com/u/5166057412?refer_flag=10010...,NaN,[]


In [0]:
Df_text_True.columns.values

array(['_position', 'comment', 'containsMusic', 'containsUrl',
       'containsVideo', 'content', 'faces', 'forward', 'hotrate',
       'isClue', 'isOrigin', 'isPiyao', 'mid', 'name', 'piclist',
       'praise', 'sourcePlatform', 'time', 'url', 'userCertify',
       'userDescription', 'userFanCount', 'userFollowCount', 'userGender',
       'userId', 'userLocation', 'userWeiboCount', 'userurl',
       'weiboLocation', 'weibo_comments'], dtype=object)

In [0]:
decision = Df_text_False.decision

In [0]:
Df_text_False.columns.values

array(['_id', 'decision', 'judgeUrl', 'reportContent', 'reportTime',
       'reportUser', 'reportUserId', 'reportedUser', 'reportedUserId',
       'reportedWeibo', 'state', 'userCertify'], dtype=object)

In [0]:
Df_text_False.reportedWeibo[0]

{'comment': '0',
 'containsUrl': False,
 'containsVideo': False,
 'faces': '',
 'forward': '0',
 'isOriginal': True,
 'piclists': ['//wx1.sinaimg.cn/thumb150/7f9efbabgy1fwtey8wcfcj20ku1127wh.jpg',
  '//wx3.sinaimg.cn/thumb150/7f9efbabgy1fwteyofwv2j20jm0dk7bv.jpg',
  '//wx1.sinaimg.cn/thumb150/7f9efbabgy1fwtey9ovhmj20qo0zkadh.jpg'],
 'praise': '0',
 'time': '2018-11-02 10:37',
 'timestamp': {'$numberLong': '1541126220000'},
 'userCertify': 0,
 'userId': '2141125547',
 'weiboContent': '#民营企业座谈会#福娃集团位于湖北省荆州市监利县的一家民营企业拖欠几千员工5-10个月不等工资，社保早已断交，据匿名网友投稿，公司扩张房地产，生态农业，与因融资失败，当地银行贷款没贷下来，已无力支付员工工资，现分地给部份员工冲抵工资……@监利新鲜事儿 @成都商报 @小鱼磨牙 @央视新闻                                            ',
 'weiboId': '4303999923956719',
 'weibourl': 'http://weibo.com/2141125547/H0Mkoyxac'}

In [0]:
content_seg = []
for i, content in enumerate(decision):
    content = content.strip()
    content = ''.join(re.findall('[\u4e00-\u9fa5]', content))
    content_temp = jieba.cut(content)
    for word in content_temp:
        content_seg.append(word)
        
import collections
cnt = collections.Counter()
for word in content_seg:
    cnt[word] += 1

cm = cnt.most_common(100)
decision_word = [i[0] for i in cm]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.230 seconds.
Prefix dict has been built succesfully.


In [0]:
#加载停用词
def getStopwords():
    stopwords = []
    with open("Dataset/stop_word_zh.txt", "r", encoding='utf8') as f:
        lines = f.readlines()
        for line in lines:
            stopwords.append(line.strip())
    return stopwords
stopwords = getStopwords()

In [0]:
content_seg = []
for i, content in enumerate(decision):
    content = content.strip()
    content = ''.join(re.findall('[\u4e00-\u9fa5]', content))
    content_temp = jieba.cut(content)
    for word in content_temp:
        if word not in stopwords and word not in decision_word:
            content_seg.append(word)

In [0]:
cnt = collections.Counter()
for word in content_seg:
    cnt[word] += 1

rumor = cnt.most_common(100)
print(rumor)

[('判定', 495), ('数据', 485), ('最新', 484), ('没有', 483), ('天津', 468), ('受伤', 460), ('无关', 459), ('实际', 453), ('已经', 453), ('中国', 450), ('孩子', 445), ('未', 433), ('网上', 423), ('言行', 411), ('一事', 408), ('系', 405), ('致人', 387), ('小女孩', 386), ('昨日', 384), ('名', 377), ('进行', 363), ('爆炸事件', 358), ('说法', 355), ('此事', 350), ('目前', 347), ('类似', 328), ('过期', 327), ('女子', 327), ('报道', 326), ('男子', 325), ('公安', 316), ('做', 314), ('图文', 311), ('嫌疑人', 309), ('造成', 301), ('流传', 283), ('抢', 280), ('对此', 273), ('并未', 260), ('照片', 260), ('媒体', 256), ('公司', 236), ('存在', 232), ('爆炸', 224), ('江苏', 223), ('找到', 219), ('地震', 219), ('走', 218), ('此类', 217), ('四川', 215), ('记者', 213), ('发现', 212), ('一起', 208), ('儿童', 206), ('医院', 205), ('犯罪', 203), ('广为流传', 199), ('案件', 196), ('票', 196), ('丢', 195), ('赣州', 192), ('人数', 189), ('乔任', 189), ('出现', 187), ('岁', 184), ('表示', 181), ('近两年', 180), ('认为', 178), ('一个', 177), ('博发', 175), ('现场', 174), ('情况', 174), ('警', 173), ('包括', 173), ('打死', 171), ('确认', 167), ('不实经', 163), 

In [0]:
Sntlst_v2 = open('Sntlst_v2.txt', 'w', encoding='utf-8')
with open('Sntlst_v2.txt', 'r', encoding='utf-8') as content:
    for i, content in enumerate(Df_text):
        content = content.strip()
        content = ''.join(re.findall('[\u4e00-\u9fa5]', content))
        content_temp = jieba.cut(content)
        for word in content_temp:
            if word not in stopwords and word not in decision_word:
                Sntlst_v2.write(word + ' ')
        Sntlst_v2.write('\n')

In [0]:
content_seg = []
for i, content in enumerate(Df_text_False.reportedWeibo):
    content = content["weiboContent"]
    content = content.strip()
    content = ''.join(re.findall('[\u4e00-\u9fa5]', content))
    content_temp = jieba.cut(content)
    for word in content_temp:
        if word not in stopwords:
            content_seg.append(word)

In [0]:
len(content_seg)

191993

In [0]:
content_seg = []
for i, content in enumerate(Df_text_True.content):
#     content = content["weiboContent"]
    content = content.strip()
    content = ''.join(re.findall('[\u4e00-\u9fa5]', content))
    content_temp = jieba.cut(content)
    for word in content_temp:
        if word not in stopwords:
            content_seg.append(word)

In [0]:
i

7906

In [0]:
TSntlst_v4 = open('./Model/Sntlst_Truth_v4.txt', 'w', encoding='utf-8')
with open('./Model/Sntlst_Truth_v4.txt', 'r', encoding='utf-8') as content:
    for i, content in enumerate(Df_text_True.content):
        content = content.strip()
        content = ''.join(re.findall('[\u4e00-\u9fa5]', content))
        content_temp = jieba.cut(content)
        for word in content_temp:
            if word not in stopwords:
                TSntlst_v4.write(word + ' ')
        TSntlst_v4.write('\n')

In [0]:
RSntlst_v4 = open('./Model/Sntlst_Rumor_v4.txt', 'w', encoding='utf-8')

with open('./Model/Sntlst_Rumor_v4.txt', 'w', encoding='utf-8') as content:
    for i, content in enumerate(Df_text_False.reportedWeibo):
        content = content["weiboContent"]
        content = content.strip()
        content = ''.join(re.findall('[\u4e00-\u9fa5]', content))
        content_temp = jieba.cut(content)
        for word in content_temp:
            if word not in stopwords:
                RSntlst_v4.write(word + ' ')
        RSntlst_v4.write('\n')

In [0]:
i

7879